In [1]:
# type: ignore

try:
  import sys
  sys.path.append('../')
  from modules.search_prop import *
except ImportError:
  url = 'https://raw.githubusercontent.com/jhjeong314/Proofmood/main/modules'
  import httpimport
  with httpimport.remote_repo(url):
    from search_prop import *

from pprint import pprint

In [2]:
# NodeLabel.build_str() test

label1 = NodeLabel(LabelType.FORMULA, 'top .')
label2 = NodeLabel(LabelType.FORMULA, 'top')
print(label1 == label2)
print()
print(label1.build_str())
print()
print(label2.build_str())
label3 = NodeLabel(LabelType.FORMULA, 'top .hyp')
label4 = NodeLabel(LabelType.FORMULA, '.hyp')
print()
print(label3 == label4)
print()
print(label3.build_str())
print()
print(label4.build_str())

True

type: formula
line: top .
formula: top
ann: 
is_hyp: False

type: formula
line: top
formula: top
ann: 
is_hyp: False

True

type: formula
line: top .hyp
formula: top
ann: hyp 
is_hyp: True

type: formula
line: .hyp
formula: top
ann: hyp 
is_hyp: True


In [3]:
# Create a blank proof.
proof = ProofNodeS() # blank proof
proof.show_fitch_text()

│1. 	 .hyp
├─
│2.  


In [4]:
# Set hypothesis.
proof.update_formula(1, Formula('A'))
proof.show_fitch_text()

│1. A	 .hyp
├─
│2.  


In [5]:
# Set conclusion.
proof.update_formula(2, Formula('B imp A'))
proof.annotate(2, Ann('imp intro 1-2'))
proof.show_fitch_text()

│1. A	 .hyp
├─
│2. B imp A	x imp intro 1-2


In [6]:
# insert a subproof
blank_proof = ProofNodeS() # blank proof to be inserted
proof.insert_node(2, blank_proof)
proof.show_fitch_text()

│1. A	 .hyp
├─
││2. 	 .hyp
│├─
││3.  
│4. B imp A	x imp intro 1-4


In [7]:
# Update the inserted subproof.
proof.update_formula('2', Formula('B'))
proof.update_formula('3', Formula('A'))
proof.annotate('3', Ann('repeat 1'))
proof.annotate('4', Ann('imp intro 2-3'))
proof.show_fitch_text()

│1. A	 .hyp
├─
││2. B	 .hyp
│├─
││3. A	✓ repeat 1
│4. B imp A	✓ imp intro 2-3


In [8]:
proof.show_fitch_text()

│1. A	 .hyp
├─
││2. B	 .hyp
│├─
││3. A	✓ repeat 1
│4. B imp A	✓ imp intro 2-3


In [9]:
import copy
prf = copy.deepcopy(proof)
prf.insert_node(3)
prf.show_fitch_text()

│1. A	 .hyp
├─
││2. B	 .hyp
│├─
││3. 
││4. A	✓ repeat 1
│5. B imp A	✓ imp intro 2-4


In [10]:
prf.delete_node(3)
prf.show_fitch_text()

│1. A	 .hyp
├─
││2. B	 .hyp
│├─
││3. A	✓ repeat 1
│4. B imp A	✓ imp intro 2-3


In [ ]:
# ProofNode.build_str() test

proof.show_fitch_text()
print()

if proof.index_dict is not None:
  for key in proof.index_dict:
    p_node = proof.get_p_node(key)
    print(p_node.build_str())
    print()


In [ ]:
# test delete_node() method

import copy
proof1 = copy.deepcopy(proof)
proof1.delete_node('1')
proof1.show_fitch_text()

In [ ]:
proof2 = copy.deepcopy(proof)
proof2.delete_node('2')
proof2.show_fitch_text()

In [ ]:
proof3 = copy.deepcopy(proof)
proof3.delete_node('3')
proof3.show_fitch_text()

In [ ]:
proof4 = copy.deepcopy(proof)
proof4.delete_node('4')
proof4.show_fitch_text()

In [ ]:
proof23 = copy.deepcopy(proof)
proof23.delete_node('2-3')
proof23.show_fitch_text()

In [ ]:
# test update_formula() method
fmla = Formula('A imp B')
proof1 = copy.deepcopy(proof)
proof1.update_formula(1, fmla)
proof1.show_fitch_text()

In [ ]:
proof2 = copy.deepcopy(proof)
proof2.update_formula(2, fmla)
proof2.show_fitch_text()

In [ ]:
proof2.update_formula(3, fmla)
proof2.annotate('3', Ann('repeat 2'))
proof2.show_fitch_text()

In [ ]:
proof2.update_formula(4, Formula('(A imp B) imp A imp B'))
proof2.show_fitch_text()

In [ ]:
proof.show_fitch_text()

In [ ]:
# insert a blank line in the conclusion
proof.insert_node(3)
proof.show_fitch_text()

In [ ]:
proof.annotate(5, Ann('imp intro 2-4'))
# or, we can use the following instead of the above.
# proof.search_proof()
proof.show_fitch_text()
# p_node2 = proof.get_p_node(2)
# print(p_node2.build_str())

In [ ]:
# insert a blank line in the hypothesis
proof.insert_node(1)
proof.show_fitch_text()

In [ ]:
# insert a comment in the hypothesis
label = NodeLabel(type=LabelType.COMMENT_HYP, 
                  line='# silly comment')
proof.insert_node(1, ProofNode(label))
proof.show_fitch_text()

In [ ]:
# Replace the blank line (Line 2) with a comment.
label = NodeLabel(type=LabelType.COMMENT_HYP, 
                  line='# blank line replaced')
new_node = ProofNode(label)
proof.replace_node(2, new_node)
proof.show_fitch_text()

In [ ]:
# Replace the blank line (Line 5) in conclusion
#  with a comment.
label = NodeLabel(type=LabelType.COMMENT_CONC, 
                  line='# comment in conclusion')
p_node = ProofNode(label)
proof.replace_node(5, p_node)                  
proof.show_fitch_text()


In [ ]:
# Let's work on the following proof.
# (B and C imp A) imp (B imp A) or (C imp A)

prf_str = """
1. B and C imp A .hyp
proves
2. C or not C
  3. C .hyp
  proves
    4. B .hyp
    proves
    5. B and C 
    6. A 
  7. B imp A 
  8. (B imp A) or (C imp A) 
  9. not C .hyp
  proves
    10. C .hyp
    proves
    11. bot 
    12. A 
  13. C imp A 
  14. (B imp A) or (C imp A) 
15. (B imp A) or (C imp A) 
"""
proof = ProofNodeS(parse_fitch(prf_str))
proof.search_proof(1)
proof.show_fitch_text()